In [1]:
import pandas as pd
import numpy as np
import string
import re
from collections import Counter

In [2]:
raw = pd.read_csv( "../data/interim/lazada_review_clean_v2.0_2.csv")
slang_words = pd.read_csv("../data/external/kbba_ID.txt",
                        sep="\t", header=None)
slang = pd.read_csv("../data/external/slangword_ID.txt",
                        sep=":", header=None)
baku_words = pd.read_csv("../data/external/katabaku_ID.txt",
                        sep="|", header=None)
baku_words.columns = [1,0]
root_words = np.array(pd.read_csv("../data/external/rootword_ID.txt",
                        sep="\n", header=None).values)
slang_words = pd.concat([slang_words, slang, baku_words])

In [3]:
slang_words.drop_duplicates(inplace=True)
slang_words = dict(zip(slang_words[0],slang_words[1]))
raw['review'] = raw['review'].apply(lambda x: x.replace(","," "))
def delete_suffix_nya(review):
    return re.sub("(?:nya|ny|y)[$|\s]"," ",review)
    
raw['review'] = raw['review'].apply(delete_suffix_nya)
raw['review'] = raw['review'].apply(lambda x: x.split())

In [4]:
def mapping_slang_words(review):
    return [slang_words[word] if word in slang_words else word for word in review]
raw['review'] = raw['review'].apply(mapping_slang_words)

In [5]:
def convert_list_to_string(word_list):
    return ",".join(word_list)
raw['review'] = raw['review'].apply(convert_list_to_string)
raw = raw.dropna(subset=['review'],how='all')
raw = raw[raw['review'].map(len) > 0]

In [6]:
def avg_word(words):
  return (sum(len(word) for word in words)/len(words))

raw['avg_word'] = raw['review'].apply(lambda x: avg_word(x.split(",")))

In [7]:
raw.to_csv("../data/interim/lazada_review_clean_v2.0_3.csv", index=False)

In [ ]:
class spellCheck:
    def train(self,features):
        model = collections.defaultdict(lambda:1)
        for f in features:
            model[f] += 1
        return model
    
    def __init__(self):
        self.NWORDS = self.train(self.words(open('../data/external/spellingset_ID.txt').read()))
        self.alphabet = 'abcdefghijklmnopqrstuvwxyz'
    
    def words(self,text):
        return re.findall('[a-z]+', text.lower())
    
    def edits1(self, word):
        splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes    = [a + b[1:] for a, b in splits if b]
        transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
        replaces   = [a + c + b[1:] for a, b in splits for c in self.alphabet if b]
        inserts    = [a + c + b     for a, b in splits for c in self.alphabet]
        return set(deletes + transposes + replaces + inserts)
    
    def known_edits2(self, word):
        return set(e2 for e1 in self.edits1(word) for e2 in self.edits1(e1) if e2 in self.NWORDS)

    def known(self,words): return set(w for w in words if w in self.NWORDS)

    def correct(self, word):
        candidates = self.known([word]) or self.known(self.edits1(word)) or self.known_edits2(word) or [word]
        return max(candidates, key=self.NWORDS.get)

def correctSpelling(text):
    sc = spellCheck()
    return sc.correct(text) if text not in root_words else text

In [ ]:
def spelling_correction(word_list):
    transformed = []
    for word in word_list:
        transformed.append(correctSpelling(word))
    return np.array(transformed)